In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

# Scrapping Data website scopus

Scraping data dari website Scopus dilakukan dengan tujuan untuk mengumpulkan informasi publikasi ilmiah yang telah diterbitkan oleh seorang penulis atau institusi tertentu. Informasi ini meliputi judul artikel, jenis dokumen, daftar penulis, nama jurnal, serta tahun dan volume publikasi.

In [ ]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Install Firefox and geckodriver
# Use !apt-get to install system packages in Colab
#!apt-get update
#!apt-get install firefox
#!apt-get install geckodriver

# Add geckodriver to PATH (might be needed if not automatically added)
import os

In [ ]:
import os
import time
import re
import csv
import json
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
import random
import pandas as pd

# os.environ["PATH"] += os.pathsep + "/usr/bin" # Ini mungkin tidak lagi diperlukan di Colab, tapi tidak ada salahnya jika ada.

url = "https://id.elsevier.com/as/authorization.oauth2?platSite=SC%2Fscopus&ui_locales=en-US&scope=openid+profile+email+els_auth_info+els_analytics_info+urn%3Acom%3Aelsevier%3Aidp%3Apolicy%3Aproduct%3Aindv_identity&els_policy=idp_policy_indv_identity_plus&response_type=code&redirect_uri=https%3A%2F%2Fwww.scopus.com%2Fauthredirect.uri%3FtxGid%3Dac07d4502027ce2ccc2422f962384f29&state=userLogin%7CtxId%3DD692FE65AD98B626728847C86FFE0CA7.i-068cc22061b9f8798%3A3&authType=SINGLE_SIGN_IN&prompt=login&client_id=SCOPUS"

options = FirefoxOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")

def main_scopus_scraper():
    driver = None
    try:
        driver = webdriver.Firefox(options=options)
        print("🚀 Memulai Scopus Scraper...")
        driver.get(url)
        time.sleep(random.uniform(3, 7))

        # --- Klik tombol "Accept Cookies" ---
        try:
            accept_cookies = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
            )
            accept_cookies.click()
            print("✅ Tombol 'Accept Cookies' diklik.")
            time.sleep(random.uniform(1, 3))
        except TimeoutException:
            print("❗ Tombol 'Accept Cookies' tidak muncul atau tidak dapat diklik dalam waktu yang ditentukan. Melanjutkan tanpa klik.")
            pass
        except Exception as e:
            print(f"❌ Error saat mencoba klik tombol cookies: {e}")
            driver.save_screenshot("cookie_error.png")


        # --- Masukkan Email ---
        input_email = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "bdd-email"))
        )
        input_email.clear()
        input_email.send_keys("nur.aiiniinuraini@apps.ipb.ac.id")
        print("✅ Email dimasukkan.")
        time.sleep(random.uniform(1, 3))

        # --- Klik tombol "Continue" setelah email ---
        continue_after_email = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='bdd-elsPrimaryBtn']"))
        )
        continue_after_email.click()
        print("✅ Tombol 'Continue' setelah email diklik.")
        time.sleep(random.uniform(5, 10))


        # --- Cek halaman setelah email (kemungkinan OTP/verifikasi) ---
        current_url_after_email = driver.current_url.lower()
        page_source_after_email = driver.page_source.lower()

        if "login" in current_url_after_email and "password" not in page_source_after_email:
            print("❗ Kemungkinan terjadi pengalihan atau memerlukan verifikasi tambahan setelah email. Tidak ada kolom password.")
            driver.save_screenshot("login_email_issue.png")
            return


        # --- Masukkan Password ---
        input_pass = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "bdd-password"))
        )
        input_pass.clear()
        input_pass.send_keys("Nuraini101#")
        print("✅ Password dimasukkan.")
        time.sleep(random.uniform(1, 3))

        # --- Klik tombol "Sign In" ---
        sign_up = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='bdd-elsPrimaryBtn']"))
        )
        sign_up.click()
        print("✅ Tombol 'Sign In' diklik.")
        time.sleep(random.uniform(7, 15))


        # --- PENTING: Pengecekan setelah login (URL dan konten) ---
        current_url_after_login = driver.current_url.lower()
        page_source_after_login = driver.page_source.lower()

        if "scopus.com" not in current_url_after_login:
            print(f"❗ Gagal login atau tidak dialihkan ke domain Scopus. Current URL: {driver.current_url}")
            driver.save_screenshot("post_login_issue_domain.png")
            return
        elif "authentication failed" in page_source_after_login or \
             "invalid username or password" in page_source_after_login or \
             "login was unsuccessful" in page_source_after_login:
            print("❗ Login gagal: Kredensial tidak valid atau masalah autentikasi.")
            driver.save_screenshot("login_failed_credentials.png")
            return
        elif "captcha" in page_source_after_login or "unusual traffic" in page_source_after_login:
            print("❗ Terdeteksi CAPTCHA atau blokir setelah login.")
            driver.save_screenshot("captcha_post_login.png")
            return
        else:
            print(f"✅ Berhasil login atau dialihkan ke Scopus. Current URL: {driver.current_url}")
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.ID, "author"))
                )
                print("✅ Elemen kunci ('Author' button) setelah login berhasil ditemukan.")
            except TimeoutException:
                print("❗ Timeout: Elemen kunci setelah login tidak ditemukan. Halaman mungkin belum selesai memuat atau struktur berubah.")
                driver.save_screenshot("post_login_element_timeout.png")
                return


        # --- Klik tombol "Authors" ---
        authors = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='author']"))
        )
        authors.click()
        print("✅ Tombol 'Author' diklik.")
        time.sleep(random.uniform(2, 4))


        # --- Masukkan Nama Belakang dan Nama Depan ---
        Last_name = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.NAME, "st1"))
        )
        Last_name.clear()
        Last_name.send_keys("Susetyo")
        print("✅ Nama belakang dimasukkan: Susetyo")
        time.sleep(random.uniform(1, 2))

        first_name = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.NAME, "st2"))
        )
        first_name.clear()
        first_name.send_keys("Budi")
        print("✅ Nama depan dimasukkan: Budi")
        time.sleep(random.uniform(3, 6))


        # --- Klik tombol "Search" ---
        try:
            # Menggunakan XPath yang Anda berikan
            search_button_locator = (By.XPATH, '//*[@id="scopus-author-search-form-experimental"]/div[3]/div[2]/button/span[1]')

            # Pastikan elemen ada di DOM
            search_button_element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located(search_button_locator)
            )

            # --- Tambahan: Tunggu hingga tombol tidak lagi disabled ---
            WebDriverWait(driver, 10).until_not(
                EC.element_attribute_to_include(search_button_locator, "disabled")
            )
            # Dan juga tunggu hingga elemen yang diwakili oleh locator menjadi clickable
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(search_button_locator)
            )

            time.sleep(random.uniform(1, 3)) # Beri sedikit jeda setelah tombol siap
            search_button_element.click() # Klik elemen yang sudah ditemukan
            print("✅ Tombol 'Search' diklik.")
            time.sleep(random.uniform(5, 10)) # Penundaan setelah klik pencarian
        except TimeoutException:
            print("❗ Timeout: Tombol 'Search' tidak ditemukan atau tidak menjadi clickable dalam waktu yang ditentukan.")
            driver.save_screenshot("search_button_timeout.png")
            return
        except NoSuchElementException:
            print("❗ NoSuchElementException: Tombol 'Search' tidak ditemukan di DOM.")
            driver.save_screenshot("search_button_no_such_element.png")
            return
        except Exception as e:
            print(f"❌ Error umum saat mengklik tombol Search: {e}")
            driver.save_screenshot("search_button_error.png")
            return


        # --- Pengecekan halaman hasil pencarian penulis ---
        if "author.uri" not in driver.current_url.lower() and "search" not in driver.current_url.lower():
            print("❗ Gagal mencapai halaman hasil pencarian penulis. Current URL:", driver.current_url)
            driver.save_screenshot("author_search_results_issue.png")
            return
        else:
            print("✅ Berhasil mencapai halaman hasil pencarian penulis.")

        # --- Klik profil penulis ---
        search_name = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//a[contains(@title, \"View this author's profile\")]"
            ))
        )
        search_name.click()
        print("✅ Profil penulis diklik.")
        time.sleep(random.uniform(5, 10))


        # --- Scroll perlahan ke bawah agar data fully loaded (optional) ---
        print("⏳ Menggulir halaman profil penulis...")
        for i in range(0, 3000, 500):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(random.uniform(0.5, 1.5))
        print("✅ Selesai menggulir halaman.")


        wait = WebDriverWait(driver, 30)


        # --- Tangani dropdown "Display" ---
        try:
            dropdown_container = wait.until(EC.presence_of_element_located((By.XPATH, "//label[.//span[text()='Display']]")))
            select_element = dropdown_container.find_element(By.CSS_SELECTOR, "select")
            select = Select(select_element)

            available_options = [o.get_attribute('value') for o in select.options]
            print(f"Opsi display yang tersedia: {available_options}")

            if '200' in available_options:
                select.select_by_value('200')
                print("✅ Memilih '200' item per halaman.")
                time.sleep(random.uniform(5, 10))
                for i in range(0, 3000, 500):
                    driver.execute_script(f"window.scrollTo(0, {i});")
                    time.sleep(random.uniform(0.5, 1.5))
            else:
                print("Opsi '200' tidak tersedia. Menggunakan pengaturan default.")
        except Exception as e:
            print(f"❌ Error saat mengatur opsi display: {e}")
            driver.save_screenshot("display_option_error.png")


        # --- Mengambil data publikasi ---
        print("⏳ Mengambil data publikasi...")
        containers = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, 'li[data-testid="results-list-item"]')
        ))

        print(f"Jumlah data ditemukan: {len(containers)}")

        data = []

        for container in containers:
            article_text = ''
            title_text = ''
            authors_text = ''
            journal_text = ''
            year_detail_text = ''
            article_url = '' # <<< Variabel untuk URL artikel

            # Article Type
            try:
                article_text = container.find_element(By.CSS_SELECTOR, 'div[data-testid="document-type-container"]').text.strip()
            except NoSuchElementException:
                pass

            # Title and Article URL
            try:
                # Cari link <a> yang membungkus judul
                title_link_element = container.find_element(By.CSS_SELECTOR, 'div.elements-module__IeohX h4 a')
                title_text = title_link_element.text.strip()
                article_url = title_link_element.get_attribute('href') # <<< Ambil atribut href
            except NoSuchElementException:
                # Jika link tidak ditemukan, coba ambil teks dari h4 saja (tanpa link)
                try:
                    title_element = container.find_element(By.CSS_SELECTOR, 'div.elements-module__IeohX h4')
                    title_text = title_element.text.strip()
                except NoSuchElementException:
                    pass
                article_url = '' # Jika tidak ada link, URL kosong


            # Authors
            try:
                author_elements = container.find_elements(By.CSS_SELECTOR, 'div[data-testid="author-list"] span.Authors-module__umR1O span')
                authors_texts = [a.text.strip() for a in author_elements if a.text.strip()]
                authors_text = ', '.join(authors_texts)
            except NoSuchElementException:
                pass

            # Journal & Year/Details (Perbaikan di sini)
            try:
                # Scopus sering memiliki struktur di mana detail jurnal/konferensi
                # dan tahun/volume ada di elemen Typography-module__...
                # Mari kita coba ambil semua teks dari elemen yang relevan di bagian detail
                # dan kemudian ekstrak tahun dari sana.

                # Mengambil semua elemen yang mungkin berisi detail jurnal/tahun
                detail_elements = container.find_elements(By.CSS_SELECTOR, 'span.Typography-module__lVnit.Typography-module__fRnrd.Typography-module__Nfgvc')

                journal_parts = []
                year_part = ''
                volume_part = ''

                for element in detail_elements:
                    text = element.text.strip()
                    if text:
                        # Coba cari tahun (4 digit)
                        year_match = re.search(r'\b(19|20)\d{2}\b', text) # Mencari tahun antara 1900-2099
                        if year_match:
                            year_part = year_match.group(0)
                            # Hapus tahun dari teks untuk menyisakan jurnal/konferensi
                            text = text.replace(year_part, '').strip()

                        # Coba cari volume/issue (seringkali dalam format angka atau V(I))
                        volume_match = re.search(r'\b(Vol\.?\s*\d+|Issue\s*\d+|V\d+\s*\(\d+\))\b', text, re.IGNORECASE)
                        if volume_match:
                            volume_part = volume_match.group(0)
                            text = text.replace(volume_part, '').strip()

                        # Sisa teks dianggap bagian dari jurnal/konferensi
                        if text:
                            journal_parts.append(text.strip('., ')) # Hapus koma/titik di akhir

                journal_text = ', '.join(filter(None, journal_parts)) # Gabungkan bagian jurnal yang ditemukan
                year_detail_text = ''
                if year_part:
                    year_detail_text += year_part
                if volume_part:
                    if year_detail_text:
                        year_detail_text += ', '
                    year_detail_text += volume_part

            except Exception as e:
                print(f"⚠ Peringatan: Gagal mengurai jurnal/tahun/volume untuk publikasi: {e}")
                journal_text = ''
                year_detail_text = ''


            data.append({
                'Article': article_text,
                'Title': title_text,
                'Authors': authors_text,
                'Journal': journal_text,
                'Year/Volume': year_detail_text,
                'Article URL': article_url # <<< Tambahkan URL ke data
            })

        df = pd.DataFrame(data)
        print("✅ Scraping Selesai. DataFrame dibuat.")
        print(df.head())
        df.to_csv("scopus_data.csv", index=False, encoding='utf-8')
        print("✅ Data disimpan ke 'scopus_data.csv'")

    except Exception as e:
        print(f"❌ Terjadi kesalahan umum selama scraping: {e}")
        if driver:
            driver.save_screenshot("general_error.png")
            print("Screenshot 'general_error.png' disimpan.")
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main_scopus_scraper()

🚀 Memulai Scopus Scraper...
✅ Tombol 'Accept Cookies' diklik.
✅ Email dimasukkan.
✅ Tombol 'Continue' setelah email diklik.
✅ Password dimasukkan.
✅ Tombol 'Sign In' diklik.
✅ Berhasil login atau dialihkan ke Scopus. Current URL: https://www.scopus.com/pages/home?display=basic#basic
✅ Elemen kunci ('Author' button) setelah login berhasil ditemukan.
✅ Tombol 'Author' diklik.
✅ Nama belakang dimasukkan: Susetyo
✅ Nama depan dimasukkan: Budi
✅ Tombol 'Search' diklik.
✅ Berhasil mencapai halaman hasil pencarian penulis.
✅ Profil penulis diklik.
⏳ Menggulir halaman profil penulis...
✅ Selesai menggulir halaman.
Opsi display yang tersedia: ['10', '20', '50', '100', '200']
✅ Memilih '200' item per halaman.
⏳ Mengambil data publikasi...
Jumlah data ditemukan: 17
✅ Scraping Selesai. DataFrame dibuat.
                   Article                                              Title  \
0                  Article  Modified tree-based selection in hierarchical ...   
1  Article  •  Open access  Improvi

Hasil data yang berhadil terambil dari Scopus ditampilkan di atas, lalu disimpan dalam file csv

# Scrapping Data website Google Scholar


Data yang akan diambil selanjutnya yaitu dari website Google Scholar dengan menggunakan API gratis. Karena website Google Scholar memiliki keamanan dengan menggunakan API, namun API yang digunakan di sini adalah API yang diambil dari SerpApi. Penggunaan API ini akan menggunakan library SerpApi yang sudah terintegrasi di dalam Python. Kemudian pengambilan data dilakukan secara berulang menggunakan fungsi “while” agar semua artikel dapat terambil secara bertahap. Sama seperti sebelumnya elemen yang akan diambil adalah judul, penulis, publikasi, tahun, link dan jurnal

In [ ]:
!pip install google-search-results
!pip install serpapi

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=4339ba2e152ee04c8ac81c418c63531590c1a204b36080eb76b59ea9d594ea00
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
from serpapi.google_search import GoogleSearch
import pandas as pd
import time

In [ ]:
# Pertama, uninstall semua versi yang mungkin konflik
!pip uninstall serpapi google-search-results -y

# Install package yang benar
!pip install google-search-results pandas

# Tunggu instalasi selesai
import time
time.sleep(3)

# Sekarang import dengan cara yang benar
from serpapi.google_search import GoogleSearch
import pandas as pd
import time

# API Key dan Author ID
api_key = "240ca5b83c3d0ccd7c7e278b2e430780135c424452e3a7a5ee7c441dbd296e53"
author_id = "apmXydQAAAAJ"

all_articles = []
start = 0
max_retries = 3  # Jumlah maksimal percobaan ulang jika gagal

while True:
    for attempt in range(max_retries):
        try:
            params = {
                "engine": "google_scholar_author",
                "author_id": author_id,
                "api_key": api_key,
                "hl": "id",
                "num": 100,
                "start": start
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            articles = results.get("articles", [])
            if not articles:
                print("✅ Semua artikel berhasil diambil.")
                break

            for article in articles:
                # ✅ Perbaikan di sini untuk "Penulis"
                authors = article.get("authors")
                if isinstance(authors, list):
                    penulis = ", ".join(authors)
                else:
                    penulis = authors if authors else "N/A"

                all_articles.append({
                    "Judul": article.get("title"),
                    "Link": article.get("link"),
                    "Tahun": article.get("year"),
                    "Penulis": penulis,
                    "Publikasi": article.get("publication", "N/A")
                })

            print(f"🔄 Halaman offset {start} - Artikel terkumpul: {len(all_articles)}")
            start += 100
            time.sleep(2)  # delay kecil untuk jaga kuota
            break

        except Exception as e:
            if attempt == max_retries - 1:
                print(f"❌ Gagal setelah {max_retries} percobaan: {str(e)}")
                break
            print(f"⚠ Percobaan {attempt + 1} gagal, mencoba lagi...")
            time.sleep(5)
    else:
        break

    if not articles:
        break

if all_articles:
    df_scholar = pd.DataFrame(all_articles)
    df_scholar = df_scholar.sort_values(by='Tahun', ascending=False)

    filename = "Google Scholar.csv"
    df_scholar.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"📁 Data disimpan ke '{filename}'")

    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
else:
    print("❌ Tidak ada artikel yang berhasil diambil.")

Found existing installation: serpapi 0.1.5
Uninstalling serpapi-0.1.5:
  Successfully uninstalled serpapi-0.1.5
Found existing installation: google_search_results 2.4.2
Uninstalling google_search_results-2.4.2:
  Successfully uninstalled google_search_results-2.4.2
  Using cached google_search_results-2.4.2-py3-none-any.whl
🔄 Halaman offset 0 - Artikel terkumpul: 100
🔄 Halaman offset 100 - Artikel terkumpul: 200
🔄 Halaman offset 200 - Artikel terkumpul: 293
✅ Semua artikel berhasil diambil.
📁 Data disimpan ke 'Google Scholar.csv'


In [ ]:
print("\n📊 5 DATA TERATAS:")
print(df_scholar.head(5))  # Angka 5 bisa diubah sesuai kebutuhan


📊 5 DATA TERATAS:
                                                                                                                                                                                  Judul  \
157                                                                                    Simulation and Empirical Studies of Long Short-Term Memory Performance to Deal with Limited Data   
164                                                                                                         THE COMPARISON OF ARIMA AND RNN FOR FORECASTING GOLD FUTURES CLOSING PRICES   
158  Fay-Herriot Model with Random Forest-Based Fixed Effect Estimation for Small Area Estimation of Per Capita Expenditure among Poor and Agricultural Workers: Will be presented at …   
168                                                                                          Evaluation of machine learning models based on household food insecurity data in Indonesia   
167                                           

Output di atas menampilkan 5 data teratas yang berhadil diambil melalui webscraper dari Google Scholar dan disimpan dalam CSV

# Scrapping Data website SINTA

Tujuannya juga sama seperti sebelumnya, melakukan scraping dnegan menggunakan library BeautifulSoup dengan membentuk fungsi untuk mengambil elemen-elemen dari web SINTA yaitu menggambil data profil penulis dan data artikelnya. Web sinta memiliki 3 jenis artikel, yaitu dari google scholar, Scopus dan Garuda

In [ ]:
pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup

# Gunakan fungsi parse_profil_sinta dari definisi
def parse_profil_sinta(html_soup):
    profil = {}
    try:
        # Nama
        nama_elem = html_soup.select_one("div.col-lg.col-md > h3 a")
        profil["Nama Lengkap"] = nama_elem.text.strip() if nama_elem else "N/A"

        # Afiliasi, Program Studi, SINTA ID
        meta_links = html_soup.select(".meta-profile a")
        profil["Afiliasi"] = meta_links[0].text.strip() if len(meta_links) > 0 else "N/A"
        profil["Program Studi"] = meta_links[1].text.strip() if len(meta_links) > 1 else "N/A"
        profil["SINTA ID"] = meta_links[2].text.strip().split(":")[-1].strip() if len(meta_links) > 2 else "N/A"

        # Skor-skor
        skor_elements = html_soup.select(".stat-profile .pr-num")
        profil["SINTA Score Overall"] = skor_elements[0].text.strip() if len(skor_elements) > 0 else "N/A"
        profil["SINTA Score 3Yr"] = skor_elements[1].text.strip() if len(skor_elements) > 1 else "N/A"
        profil["Affil Score"] = skor_elements[2].text.strip() if len(skor_elements) > 2 else "N/A"
        profil["Affil Score 3Yr"] = skor_elements[3].text.strip() if len(skor_elements) > 3 else "N/A"

        # Tabel Statistik (Scopus, GScholar)
        metric_table = html_soup.select_one(".stat-table tbody")
        if metric_table:
            for row in metric_table.find_all("tr"):
                cells = row.find_all("td")
                if len(cells) >= 3:
                    key = cells[0].text.strip()
                    profil[f"{key} (Scopus)"] = cells[1].text.strip()
                    profil[f"{key} (GScholar)"] = cells[2].text.strip()

    except Exception as e:
        print(f"⚠ Error parsing profil: {e}")
    return profil

# Ambil HTML profil penulis
author_id = "6173018"
url = f"https://sinta.kemdikbud.go.id/authors/profile/{author_id}"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Panggil fungsi profil
profil_data = parse_profil_sinta(soup)

# Tampilkan hasilnya
print("\n=== PROFIL PENULIS SINTA ===")
for k, v in profil_data.items():
    print(f"{k}: {v}")



=== PROFIL PENULIS SINTA ===
Nama Lengkap: BUDI SUSETYO
Afiliasi: Institut Pertanian Bogor
Program Studi: S2 - Statistika dan Sains Data
SINTA ID: 6173018
SINTA Score Overall: 793
SINTA Score 3Yr: 257
Affil Score: 793
Affil Score 3Yr: 257
Article (Scopus): 17
Article (GScholar): 100
Citation (Scopus): 16
Citation (GScholar): 335
Cited Document (Scopus): 11
Cited Document (GScholar): 61
H-Index (Scopus): 2
H-Index (GScholar): 9
i10-Index (Scopus): 0
i10-Index (GScholar): 9
G-Index (Scopus): 1
G-Index (GScholar): 1


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time # Import modul time untuk jeda

def parse_profil_sinta(html_soup):
    profil = {}
    try:
        profil["Nama Lengkap"] = html_soup.select_one("div.col-lg.col-md > h3 a").text.strip()
        profil["Afiliasi"] = html_soup.select_one(".meta-profile a:nth-child(1)").text.strip()
        profil["Program Studi"] = html_soup.select_one(".meta-profile a:nth-child(3)").text.strip()
        profil["SINTA ID"] = html_soup.select_one(".meta-profile a:nth-child(5)").text.split(":")[-1].strip()

        skor_elements = html_soup.select(".stat-profile .pr-num")
        profil["SINTA Score Overall"] = skor_elements[0].text.strip() if len(skor_elements) > 0 else "N/A"
        profil["SINTA Score 3Yr"] = skor_elements[1].text.strip() if len(skor_elements) > 1 else "N/A"
        profil["Affil Score"] = skor_elements[2].text.strip() if len(skor_elements) > 2 else "N/A"
        profil["Affil Score 3Yr"] = skor_elements[3].text.strip() if len(skor_elements) > 3 else "N/A"

        metric_table = html_soup.select_one(".stat-table tbody")
        if metric_table:
            for row in metric_table.find_all("tr"):
                cells = row.find_all("td")
                if len(cells) >= 3:
                    key = cells[0].text.strip()
                    profil[f"{key} (Scopus)"] = cells[1].text.strip()
                    profil[f"{key} (GScholar)"] = cells[2].text.strip()
    except Exception as e:
        print(f"⚠ Error parsing profil: {e}")
    return profil

def scrape_sinta_selected_views(author_id, timeout=45, max_retries=5, delay_between_views=3): # Timeout dinaikkan, jeda antar view
    base_url = f"https://sinta.kemdikbud.go.id/authors/profile/{author_id}"
    views = ['googlescholar', 'scopus', 'garuda']
    all_data = []
    profil_data = {}

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7"
    }

    # --- Bagian untuk mengambil profil utama dengan retry ---
    for attempt in range(max_retries):
        try:
            print(f"Percobaan {attempt + 1}/{max_retries} untuk mengambil halaman profil utama...")
            profil_response = requests.get(base_url, headers=headers, timeout=timeout)
            profil_response.raise_for_status()
            soup_profil = BeautifulSoup(profil_response.text, 'html.parser')
            profil_data = parse_profil_sinta(soup_profil)
            print("✅ Profil utama berhasil diambil.")
            break
        except requests.exceptions.Timeout:
            print(f"⚠️ Timeout saat mengambil halaman profil utama (Percobaan {attempt + 1}/{max_retries}). Mencoba lagi...")
        except requests.exceptions.RequestException as e:
            print(f"⚠ Gagal mengambil halaman profil utama (Percobaan {attempt + 1}/{max_retries}): {e}")

        if attempt < max_retries - 1:
            time.sleep(2 ** attempt)
        else:
            print("❗ Maksimal percobaan tercapai untuk halaman profil. Tidak dapat mengambil profil.")
    # --- Akhir bagian profil utama ---

    # --- Bagian untuk mengambil data dari masing-masing view publikasi dengan retry ---
    for view in views:
        url = f"{base_url}/?view={view}"
        print(f"\n📥 Mengambil data dari: {view.upper()} - {url}")

        # Tambahkan jeda singkat sebelum memulai scraping untuk view baru
        if delay_between_views > 0:
            print(f"Menunggu {delay_between_views} detik sebelum mengambil view {view.upper()}...")
            time.sleep(delay_between_views)

        for attempt in range(max_retries):
            try:
                print(f"Percobaan {attempt + 1}/{max_retries} untuk mengambil data dari {view.upper()}...")
                response = requests.get(url, headers=headers, timeout=timeout)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')
                print(f"✅ Data {view.upper()} berhasil diambil.")

                articles = soup.select('div.ar-list-item')
                if not articles:
                    print(f"Tidak ada artikel ditemukan untuk view '{view}'.")
                    break

                for article in articles:
                    try:
                        title_elem = article.select_one('div.ar-title a')
                        title = title_elem.get_text(strip=True) if title_elem else "N/A"
                        link = title_elem['href'] if title_elem else "N/A"

                        authors_elem = article.select_one('div.ar-meta a')
                        authors = ["N/A"]
                        if authors_elem and 'Authors :' in authors_elem.get_text():
                            authors_text = authors_elem.get_text(strip=True).replace('Authors :', '').strip()
                            authors = [a.strip() for a in authors_text.split(',')]

                        journal_elem = article.select_one('a.ar-pub')
                        journal = journal_elem.get_text(strip=True).replace('\xa0', ' ').strip() if journal_elem else "N/A"
                        journal = re.sub(r',\s*(\d{4})$', '', journal)

                        # Perbaikan untuk masalah 'NoneType' object has no attribute 'group'
                        year = "N/A"
                        if article.select_one('a.ar-year'): # Coba ambil dari elemen tahun yang spesifik dulu
                            year_match = re.search(r'\d{4}', article.select_one('a.ar-year').get_text(strip=True))
                            if year_match:
                                year = year_match.group(0)

                        # Fallback jika tahun tidak ditemukan di a.ar-year atau jika year_elem tidak ada
                        if year == "N/A" and journal_elem:
                            year_match_journal = re.search(r'\d{4}', journal_elem.get_text(strip=True))
                            if year_match_journal:
                                year = year_match_journal.group(0)

                        cited_elem = article.select_one('a.ar-cited')
                        cited = re.search(r'\d+', cited_elem.get_text(strip=True)).group(0) if cited_elem and re.search(r'\d+', cited_elem.get_text(strip=True)) else "0"


                        all_data.append({
                            'Sumber': view.upper(),
                            'Judul': title,
                            'Penulis': '; '.join(authors),
                            'Jurnal': journal,
                            'Tahun': year,
                            'Cited': cited,
                            'Link': link
                        })
                    except Exception as e:
                        print(f"⚠ Error saat memproses artikel pada view '{view}': {str(e)}")
                        continue
                break

            except requests.exceptions.Timeout:
                print(f"⚠️ Timeout saat mengambil data '{view}' (Percobaan {attempt + 1}/{max_retries}). Mencoba lagi...")
            except requests.exceptions.RequestException as e:
                print(f"Request error untuk view '{view}' (Percobaan {attempt + 1}/{max_retries}): {str(e)}")

            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print(f"❗ Maksimal percobaan tercapai untuk view '{view}'. Tidak dapat mengambil data.")
    # --- Akhir bagian view publikasi ---

    return profil_data, pd.DataFrame(all_data)


if __name__ == "__main__":
    author_id = "6173018"
    print(f"Memulai scraping untuk SINTA ID: {author_id}")
    profil, df_artikel = scrape_sinta_selected_views(author_id)

    print("\n---")
    print("📄 Profil Penulis:")
    if profil:
        for key, value in profil.items():
            print(f"{key}: {value}")
    else:
        print("Profil penulis tidak berhasil diambil.")

    print("\n---")
    if df_artikel.empty:
        print("Tidak ada data artikel yang berhasil diambil.")
    else:
        print(f"📑 Total Artikel Berhasil Diambil: {len(df_artikel)}")
        print("\n5 Artikel Pertama:")
        print(df_artikel.head())

        try:
            df_artikel.to_csv(f"sinta_articles_{author_id}.csv", index=False, encoding='utf-8')
            # Pastikan profil_data tidak kosong sebelum menyimpan
            if profil:
                pd.DataFrame([profil]).to_csv(f"sinta_profile_{author_id}.csv", index=False, encoding='utf-8')
            else:
                print("Tidak dapat menyimpan profil karena kosong.")
            print(f"\n✅ Data profil dan artikel disimpan ke file CSV.")
        except Exception as e:
            print(f"❌ Gagal menyimpan file CSV: {e}")

    print("\n--- Selesai ---")

Memulai scraping untuk SINTA ID: 6173018
Percobaan 1/5 untuk mengambil halaman profil utama...
✅ Profil utama berhasil diambil.

📥 Mengambil data dari: GOOGLESCHOLAR - https://sinta.kemdikbud.go.id/authors/profile/6173018/?view=googlescholar
Menunggu 3 detik sebelum mengambil view GOOGLESCHOLAR...
Percobaan 1/5 untuk mengambil data dari GOOGLESCHOLAR...
✅ Data GOOGLESCHOLAR berhasil diambil.

📥 Mengambil data dari: SCOPUS - https://sinta.kemdikbud.go.id/authors/profile/6173018/?view=scopus
Menunggu 3 detik sebelum mengambil view SCOPUS...
Percobaan 1/5 untuk mengambil data dari SCOPUS...
✅ Data SCOPUS berhasil diambil.

📥 Mengambil data dari: GARUDA - https://sinta.kemdikbud.go.id/authors/profile/6173018/?view=garuda
Menunggu 3 detik sebelum mengambil view GARUDA...
Percobaan 1/5 untuk mengambil data dari GARUDA...
✅ Data GARUDA berhasil diambil.

---
📄 Profil Penulis:
Nama Lengkap: BUDI SUSETYO
Afiliasi: Institut Pertanian Bogor
Program Studi: S2 - Statistika dan Sains Data
SINTA ID: 

Maka data berhasil diambil dan disimpan dalam bentuk csv